## Setup

In [79]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [80]:
downloaded = drive.CreateFile({'id':'1jaSDNP_37ytHtj1moy7M48vH9heB75BQ'}) # replace the id with id of file you want to access
downloaded.GetContentFile('data.csv') 
downloaded = drive.CreateFile({'id':'1dVZhRrg2Tdw7gbnCdc-Q0wC3B03cKj2w'}) # replace the id with id of file you want to access
downloaded.GetContentFile('symptom.csv') 
downloaded = drive.CreateFile({'id':'1JRDtdihnf7mvgvT04X1FCqFJ8abg-zaU'}) # replace the id with id of file you want to access
downloaded.GetContentFile('vac.csv') 

In [81]:
import pandas as pd
patient_history=pd.read_csv('data.csv', sep=',', encoding='ISO-8859-1')
patient_sympt=pd.read_csv('symptom.csv', sep=',',encoding='ISO-8859-1')
patient_vac=pd.read_csv('vac.csv', sep=',',encoding='ISO-8859-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
pd.set_option("display.max_rows", 1000)

In [83]:
patient_history.head(2)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/28/2020,12/30/2020,2.0,None,PVT,NaN,None,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/31/2020,12/31/2020,0.0,NaN,SEN,NaN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy"""


## Data preprocessing

In [84]:
 # get the needed columns from patient history file
patient_history=patient_history.loc[: , ['VAERS_ID','AGE_YRS','SEX','CUR_ILL','HISTORY','ALLERGIES']]
 # get the needed columns from patient vaccine file
patient_vac=patient_vac.loc[:,['VAERS_ID','VAX_NAME']]
 # commbining reactions per patient 
patient_sympt['SYMPTOMS']= patient_sympt['SYMPTOM1'].fillna('')+','+ patient_sympt['SYMPTOM2'].fillna('')+','+ patient_sympt['SYMPTOM3'].fillna('')+','+ patient_sympt['SYMPTOM4'].fillna('')+','+ patient_sympt['SYMPTOM5'].fillna('')
patient_sympt=patient_sympt.groupby(by='VAERS_ID')['SYMPTOMS'].agg(lambda x: ','.join(x)).reset_index()


In [85]:
 # mergeing all extracted info above to a single dataframe 
#df=pd.merge(patient_history, patient_vac, left_on='VAERS_ID', right_on='VAERS_ID')
df=patient_history.merge( patient_vac , on='VAERS_ID')


In [86]:
df=pd.merge(df, patient_sympt,  left_on='VAERS_ID', right_on='VAERS_ID')
df=df.fillna(value="")
# prepare y values step1 
# make symptom columns into a list 
df['SYMPTOMS']=df['SYMPTOMS'].apply(lambda x : x.split(","))
# and than remove any empty values in the list. 
df['SYMPTOMS']=df['SYMPTOMS'].apply(lambda x : [i for i in x if i])
df=df.drop_duplicates(['VAERS_ID'])
df[df['VAERS_ID']==1015465]

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS
22431,1015465,75,F,U07.1 2019-nCoV acute respiratory disease (Pri...,U07.1 2019-nCoV acute respiratory disease (Pri...,"Iodinated Contrast- Oral and IV Dye, Penicilli...",COVID19 (COVID19 (MODERNA)),"[Heart rate abnormal, Pulse absent, Respirator..."


In [87]:
def convertAge(string_age):
    if string_age =="" :
        return "unknown_age"
    if float(string_age) <= 60 :
        return 'under_60'
    return "over_60"

def changeVaxName(vax_name):
    if 'MODERNA'in vax_name:
        return 'MODERNA'
    if 'PFIZER' in vax_name:
        return 'PFIZER'
    
    return 'JANSSEN'

## Dataset after preprocessing

In [88]:
# remove none coivd related vaccine samples 
df=df[df['VAX_NAME'].str.contains('|'.join(['MODERNA','JANSSEN','PFIZER-BIONTECH'])) & df['VAX_NAME'].isna()==False]
# put age in bucket under 60 or over
df['AGE_CAT']=df['AGE_YRS'].apply(lambda x : convertAge(x) )
# simmplify vaccine names 
df['VAX_NAME']=df['VAX_NAME'].apply(lambda x : changeVaxName(x))
dfS

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT
0,916600,33,F,None,None,Pcn and bee venom,MODERNA,"[Dysphagia, Epiglottitis]",under_60
1,916601,73,F,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""Dairy""",MODERNA,"[Anxiety, Dyspnoea]",over_60
2,916602,23,F,None,None,Shellfish,PFIZER,"[Chest discomfort, Dysphagia, Pain in extremit...",under_60
3,916603,58,F,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",MODERNA,"[Dizziness, Fatigue, Mobility decreased]",under_60
4,916604,47,F,Na,,Na,MODERNA,"[Injection site erythema, Injection site pruri...",under_60
...,...,...,...,...,...,...,...,...,...
34625,1113917,88,F,,Medical History/Concurrent Conditions: Aortic ...,,MODERNA,"[Dysphagia, Hernia, Obstruction]",over_60
34626,1113920,83,F,,Medical History/Concurrent Conditions: Atrial ...,,PFIZER,"[Arrhythmia, Asymptomatic COVID-19, Chills, Co...",over_60
34627,1113963,59,F,,Medical History/Concurrent Conditions: Type II...,,PFIZER,"[Haemophagocytic lymphohistiocytosis, SARS-CoV...",under_60
34628,1115045,57,M,,Medical History/Concurrent Conditions: No adve...,,MODERNA,[Death],under_60


## Extract the needed columns for training

In [89]:
df_training = df.loc[:, ['AGE_CAT','SEX','VAX_NAME']]
# combining patients' current illness , medical history and, allergies and store in the "health condition column"
df_training['HEALTH_CONDITION'] = df['CUR_ILL']+","+ df['HISTORY']+","+df['ALLERGIES']
# make the heath_condition text lower case 
df_training['HEALTH_CONDITION']=df_training['HEALTH_CONDITION'].str.lower()

In [90]:
df_training

,AGE_CAT,SEX,VAX_NAME,HEALTH_CONDITION
0,under_60,F,MODERNA,"none,none,pcn and bee venom"
1,over_60,F,MODERNA,patient residing at nursing facility. see pati...
2,under_60,F,PFIZER,"none,none,shellfish"
3,under_60,F,MODERNA,"kidney infection,diverticulitis, mitral valve ..."
4,under_60,F,MODERNA,"na,,na"
...,...,...,...,...
34625,over_60,F,MODERNA,",medical history/concurrent conditions: aortic..."
34626,over_60,F,PFIZER,",medical history/concurrent conditions: atrial..."
34627,under_60,F,PFIZER,",medical history/concurrent conditions: type i..."
34628,under_60,M,MODERNA,",medical history/concurrent conditions: no adv..."


In [91]:
wordcount=df_training['HEALTH_CONDITION'].str.split('[\,\ ]',expand=True).stack().value_counts().to_frame()

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
# define stop words 
addition_stopwords= ['none','None','na','nkda','00', '000', '000mg', '001d', '002d', '008s', '00x0', '01', '010', '0107', '010d', '011', '012', '02', '0200', '021', '029', '02jan2021', '02xa', '03', '0350', '03jan2021', '04', '0400', '041d', '0430', '04jan2021', '04nov2020', '05', '053', '06', '06jan2021', '07', '070s', '08', '080d', '080s', '0845', '08jun', '09', '091', '09nov2020', '0p', '10', '100', '1000', '100mg', '101', '102', '103', '104', '106', '10mg', '10th', '10yrs', '11', '110', '111', '112', '113', '115', '116', '119', '11jan2021', '11th', '12', '120', '1200mg', '1201005', '120320', '120ml', '121', '123', '1245', '125', '125mg', '1273', '129', '12dec2020', '12h', '12mm', '12th', '12xa', '12xd', '13', '130', '135', '13645005', '13apr2016', '13jan2021', '13nov2020', '13th', '14', '140', '142', '142d', '143', '14feb2021', '14th', '15', '1500', '1500mg', '152', '155mg', '15mg', '15min', '15pm', '15th', '16', '160cm', '160mg', '161', '16years', '17', '170s', '17feb2021', '17th', '18', '180', '1800', '18jan2021', '18mm', '18th', '19', '191813001', '192020', '193031009', '1940', '1943', '1944', '1950', '1957', '1960', '1960s', '1965', '1969', '1972', '1973', '1974', '1976', '1977', '1980', '1980s', '1983', '1985', '1986', '1987', '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '19dec2020', '19feb2021', '19jan2021', '19nov2020', '19th', '1a', '1month', '1onth', '1st', '1week', '1wk', '1x', '20', '200', '2000', '2000s', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '200mg', '201', '2010', '2010h', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '202', '2020', '2021', '20210a', '2022', '203', '204', '205', '209', '20lb', '20mg', '20nov2020', '20s', '20th', '21', '210', '212', '215q', '21dec2020', '21st', '22', '220', '222a', '229', '22dec2020', '22ndhad', '22oct2020', '23', '23dec2020', '24', '24dec2020', '24h', '24th', '25', '250', '2510', '25jan2021', '25mg', '26', '267434003', '26th', '27', '272', '274769005', '279039007', '27dec2020', '27th', '28', '285', '286', '28dec2020', '28mg', '28th', '29', '2921', '293', '298', '299', '29iu', '29nov2020', '29th', '2b', '2cm', '2dm', '2l', '2nd', '2ndary', '2o13', '2weeks', '2x', '2yrs', '30', '300', '3019', '302d', '303', '307', '309', '30mg', '30min', '30mm', '30pm', '30th', '30xa', '30xs', '31', '310', '311', '3112', '315', '318', '319', '31dec2020', '31st', '32', '320', '321', '3222', '324', '33', '34', '345', '35', '354', '359', '36', '367399005', '37', '37am', '38', '39', '396275006', '3a', '3b', '3cm', '3kg', '3l', '3mg', '3mm', '3months', '3rd', '3weeks', '3x', '40', '400', '400047006', '401', '40mg', '40s', '41', '412a', '413', '417k', '42', '421326000', '422166005', '423263001', '42xd', '43', '430pm', '433144002', '44', '440', '443', '445d', '448', '45', '45mg', '46', '462', '465', '47', '48', '49', '491', '493', '4g', '4th', '4x', '4years', '50', '500', '500mg', '501d', '507', '50s', '51', '511', '511a', '512', '512a', '515d', '52', '521', '522', '53', '536', '5370000', '53mg', '54404000', '55', '550', '551', '555', '559', '56', '561', '562', '564', '569', '57', '571', '58', '585', '59', '5cm', '5ft', '5g', '5other', '5q', '5th', '5weeks', '5wks', '5x', '5x0s', '5x9d', '5y', '5yrs', '60', '609', '61', '62', '621', '622', '63', '630pm', '64', '640', '641', '65', '650mg', '66', '67', '671', '68', '68yo', '69', '6dec2020', '6g', '6mg', '6th', '6x', '70', '71', '710', '718', '719', '72', '723', '724', '724698009', '73', '73430006', '73438004', '74', '75', '750', '758', '75mg', '76', '77', '78', '780', '79', '791', '79k', '7th', '80', '800', '800mg', '802d', '80s', '81', '810', '811', '812d', '813', '815', '816', '818', '819', '81mg', '82', '82020', '821a', '828', '829', '83', '838', '84', '841', '85', '8517006', '8580', '86', '87', '875', '88', '89', '890', '891', '898', '899', '89xs', '8am', '8cm', '8h', '8th', '90', '901', '909', '90s', '91', '911', '912', '92', '920', '921', '922', '92814006', '929', '93', '94', '95', '951', '95mg', '96', '97', '98', '99', '991', '998', '9chronic', '9in', '9mm', '9th', 'a1', 'a1c', 'a3', 'a41', 'a46', 'age24', 'alpha1', 'alpha1antitrypsin', 'aox3', 'appendectomy1988', 'apr2020', 'b00', 'b12', 'b19', 'b27', 'b35', 'b91', 'b95', 'b95a', 'b96', 'bipolar1', 'bipolar2', 'bnt162b2', 'brca2', 'c18', 'c25', 'c3', 'c4', 'c44', 'c5', 'c50', 'c53', 'c6', 'c61', 'c677t', 'c7', 'c8', 'c82', 'c85', 'c91', 'cabgx3', 'cd4', 'ckd2', 'ckd3', 'ckd4', 'cov2', 'covid10', 'covid19', 'covid_19', 'd05', 'd12', 'd3', 'd46', 'd49', 'd50', 'd51', 'd53', 'd61', 'd63', 'd64', 'd68', 'd69', 'd72', 'dec2017', 'dec2019', 'dec2020', 'dg6', 'diabetes2', 'diagnosis1', 'dissection2016', 'dm1', 'dm11', 'dm2', 'dmt2', 'dose1', 'e00', 'e02', 'e03', 'e04', 'e06', 'e07', 'e08', 'e10', 'e11', 'e26', 'e27', 'e3', 'e43', 'e46', 'e53', 'e55', 'e56', 'e63', 'e66', 'e73', 'e78', 'e86', 'e87', 'e876', 'e88', 'ees400', 'ef15', 'el0140', 'en6203', 'f01', 'f02', 'f03', 'f05', 'f06', 'f10', 'f17', 'f20', 'f22', 'f25', 'f29', 'f31', 'f32', 'f33', 'f34', 'f39', 'f41', 'f43', 'f44', 'f48', 'f51', 'f73', 'f89', 'f90', 'f95', 'f98', 'feb2020', 'for2', 'fracture11', 'ft4', 'g2', 'g20', 'g25', 'g30', 'g31', 'g35', 'g3a', 'g40', 'g43', 'g44', 'g45', 'g47', 'g56', 'g60', 'g62', 'g6pd', 'g80', 'g81', 'g82', 'g89', 'g93', 'g95', 'h04', 'h1', 'h16', 'h1c', 'h1n1', 'h25', 'h26', 'h35', 'h40', 'h47', 'h52', 'h53', 'h5n1', 'h61', 'h91', 'hba1c', 'her2', 'hga1c', 'hgba1c', 'hsv1', 'hsv2', 'hypertensionr26', 'i05', 'i07', 'i10', 'i11', 'i13', 'i16', 'i21', 'i25', 'i26', 'i27', 'i42', 'i44', 'i45', 'i47', 'i48', 'i49', 'i50', 'i51', 'i63', 'i66', 'i67', 'i69', 'i70', 'i71', 'i72', 'i73', 'i82', 'i83', 'i87', 'i89', 'i95', 'icd10', 'icd9', 'if_covid_prior_vaccination', 'igg4', 'insulin16', 'j01', 'j02', 'j15', 'j18', 'j20', 'j30', 'j31', 'j40', 'j42', 'j43', 'j44', 'j45', 'j69', 'j96', 'j98', 'jak2', 'jan2020', 'jan2021', 'jul2018', 'jul2019', 'jul2020', 'jun2018', 'jun2020', 'k21', 'k29', 'k30', 'k42', 'k43', 'k44', 'k52', 'k55', 'k56', 'k57', 'k58', 'k59', 'k62', 'k63', 'k76', 'k80', 'k85', 'k92', 'known_allergies', 'l02', 'l03', 'l08', 'l1', 'l2', 'l21', 'l23', 'l29', 'l3', 'l30', 'l4', 'l5', 'l53', 'l5s1', 'l5s4', 'l70', 'l80', 'l89', 'l97', 'l98', 'm05', 'm06', 'm10', 'm13', 'm15', 'm17', 'm19', 'm1a', 'm20', 'm24', 'm25', 'm41', 'm43', 'm47', 'm48', 'm51', 'm53', 'm54', 'm62', 'm6us', 'm79', 'm81', 'm85', 'm86', 'mar2020', 'may2020', 'medical_history', 'mm3', 'mso4', 'n0', 'n02', 'n04', 'n147', 'n17', 'n18', 'n20', 'n28', 'n31', 'n32', 'n39', 'n40', 'n62', 'n63', 'n90', 'n92', 'n95', 'nov020', 'nov18', 'nov2019', 'nov2020', 'o2', 'oct2020', 'october2020', 'om1', 'on12', 'on2', 'other_medical_history', 'p2', 'pcv13', 'polyethylene_glycol', 'pt1cn1am0', 'pt2n2am0', 'q21', 'q24', 'q38', 'q44', 'q4hrs', 'q61', 'q66', 'r00', 'r05', 'r06', 'r09', 'r13', 'r14', 'r19', 'r20', 'r23', 'r25', 'r26', 'r27', 'r29', 'r30', 'r31', 'r33', 'r41', 'r42', 'r44', 'r45', 'r47', 'r48', 'r49', 'r50', 'r52', 'r53', 'r54', 'r55', 'r56', 'r60', 'r62', 'r63', 'r73', 'r74', 'r77', 'r78', 'r79', 'r80', 'r93', 'r94', 's06', 's1', 's22', 's28', 's32', 's42', 's60', 's61', 's63', 's70', 's72', 's80', 's81', 's82', 's90', 's91', 'sep2020', 'sm859', 'sm869', 'sp02', 'spo2', 'surgery4', 't1', 't10', 't11', 't12', 't1b2', 't1dm', 't2', 't2dm', 't3', 't3n0', 't4', 't45', 't50', 't7', 't8', 't81', 't9', 'testx2', 'tiax2', 'type1', 'type2', 'type2dm', 'u07', 'v00', 'v15', 'v2', 'w19', 'x1', 'x10', 'x2', 'x20', 'x3', 'x3mo', 'x4', 'x4v', 'x50', 'x5yd', 'x6', 'x7', 'x9', 'y65', 'z11', 'z16', 'z20', 'z23', 'z29', 'z41', 'z43', 'z45', 'z46', 'z48', 'z51', 'z66', 'z68', 'z72', 'z74', 'z79', 'z83', 'z85', 'z86', 'z87', 'z90', 'z91', 'z95', 'z96', 'z97', 'z98', 'z99']
stop_w=text.ENGLISH_STOP_WORDS.union(addition_stopwords)


In [93]:
MODERNA=df_training[df_training['VAX_NAME']=='MODERNA']
PFIZER=df_training[df_training['VAX_NAME']=='PFIZER']
JANSSEN=df_training[df_training['VAX_NAME']=='JANSSEN']

In [94]:
MODERNA

,AGE_CAT,SEX,VAX_NAME,HEALTH_CONDITION
0,under_60,F,MODERNA,"none,none,pcn and bee venom"
1,over_60,F,MODERNA,patient residing at nursing facility. see pati...
3,under_60,F,MODERNA,"kidney infection,diverticulitis, mitral valve ..."
4,under_60,F,MODERNA,"na,,na"
5,under_60,F,MODERNA,",,iodine (shellfish) has epipen"
...,...,...,...,...
34623,over_60,F,MODERNA,",medical history/concurrent conditions: covid-..."
34624,over_60,M,MODERNA,",medical history/concurrent conditions: no adv..."
34625,over_60,F,MODERNA,",medical history/concurrent conditions: aortic..."
34628,under_60,M,MODERNA,",medical history/concurrent conditions: no adv..."


In [95]:
# make df_training data to sparse matrix
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_w)
tfidfs = tfidf_vectorizer.fit_transform(MODERNA['HEALTH_CONDITION'])
tfidfs = pd.DataFrame(tfidfs.toarray(), columns=tfidf_vectorizer.get_feature_names())
dummie_age_sex=pd.get_dummies(MODERNA[['SEX','AGE_CAT']],  prefix='', prefix_sep='').reset_index()
moderna_df=pd.concat([dummie_age_sex,tfidfs], axis=1)


In [96]:
moderna_df

,index,F,M,U,over_60,under_60,unknown_age,aaa,aasthma,ab,abcess,abd,abdomen,abdomenal,abdominal,abductus,abilify,ability,abiltizing,ablated,ablation,ablative,able,abnl,abnormal,abnormalities,abnormality,abo,abrasion,abrupt,abscess,abscessus,absence,absent,absolutely,absorption,abt,abuse,abx,ac,...,yesterday,ylang,yolk,yom,young,yrs,zanaflex,zantac,zero,zestril,zetia,ziacam,zinc,zithromax,zithromycin,zocar,zocin,zocor,zofran,zoledronic,zoloft,zolpidem,zomax,zomig,zone,zonegran,zonisamide,zonismade,zostavax,zoster,zosyn,zpac,zpak,zucchini,zuccini,zyban,zyprexa,zyrtec,zythromax,zythromycin
0,0,1,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16381,34623,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16382,34624,0,1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16383,34625,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16384,34628,0,1,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=200)
kmeans.fit(moderna_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=200, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [117]:
kmeans.labels_

array([180, 180, 180, ..., 117, 117, 117], dtype=int32)

In [118]:
MODERNA_CLUSTERED=df[df['VAX_NAME']=='MODERNA'].copy()

MODERNA_CLUSTERED['CLUSTER'] = kmeans.labels_

## How many instance are belonging to each cluster

In [129]:
MODERNA_CLUSTERED.groupby(by='CLUSTER')['VAERS_ID'].agg('count').sort_values()

CLUSTER
74      21
0       24
155     30
187     31
162     31
186     32
177     44
93      44
27      44
124     45
178     49
10      49
169     50
94      52
191     53
62      53
14      54
181     55
165     55
103     56
156     56
49      58
192     58
182     60
108     60
149     61
160     62
87      63
136     63
18      63
121     64
46      64
73      64
48      65
42      65
123     66
84      66
131     66
20      67
76      67
72      68
119     68
85      68
133     69
197     69
112     69
148     69
30      70
142     70
28      71
44      72
174     72
32      72
95      72
107     72
151     72
157     73
63      73
61      73
198     74
106     75
60      76
92      76
146     76
11      76
88      77
34      77
130     77
164     78
140     78
81      78
77      78
55      78
189     79
2       79
166     79
22      79
7       79
126     79
190     79
67      80
172     80
26      80
24      81
21      81
193     81
167     81
6       81
15      81
143     81
11

## what are in cluster 90 

In [146]:
sym_freq=MODERNA_CLUSTERED[MODERNA_CLUSTERED['CLUSTER']==90]
sym_freq

,VAERS_ID,AGE_YRS,SEX,CUR_ILL,HISTORY,ALLERGIES,VAX_NAME,SYMPTOMS,AGE_CAT,CLUSTER
34008,1095125,85,M,unknown,"Pancreatic cancer, HTN, CAD, afib","ACE inhibitors, iodinated contrast media, iodi...",MODERNA,"[Asthenia, Diarrhoea, Dizziness, Hepatic lesio...",over_60,90
34011,1095160,93,F,,,,MODERNA,[Death],over_60,90
34012,1095170,77,F,COPD,COPD,NKDA,MODERNA,"[Acute myocardial infarction, Asthenia, Death,...",over_60,90
34018,1095212,78,M,,Type 2 diabetes mellitus; pulmonary bronchiect...,No known allergies,MODERNA,"[Angiogram pulmonary abnormal, Aortic dilatati...",over_60,90
34019,1095232,79,F,,,steroid creams,MODERNA,"[Acute myocardial infarction, Antiplatelet the...",over_60,90
34021,1095238,69,M,"covid positive 9/14/20, recently worsening of ...","sarcoidosis, COPD, diabetes, HTN, congestive h...",none,MODERNA,"[Cardiac failure congestive, Condition aggrava...",over_60,90
34025,1095300,56,F,not known,not known,nka,MODERNA,"[Chest pain, Death, Headache, Nausea, Pain]",under_60,90
34031,1095393,76,M,None,Coronary Artery Disease,NKDA,MODERNA,"[Cardiac arrest, Toxicologic test]",over_60,90
34032,1095406,79,M,,,,MODERNA,[Death],over_60,90
34034,1095435,51,F,Feeling nauseated vomiting with headache,None,,MODERNA,"[Autopsy, Death, Dyspnoea, Headache, Nausea, P...",under_60,90


## Symptoms frequency in cluster 90 

In [147]:
from itertools import chain
from collections import Counter
pd.Series(Counter(chain.from_iterable(sym_freq.SYMPTOMS.dropna()))).sort_values(ascending=False)

Death                                               28
Dyspnoea                                            12
Fatigue                                             11
Nausea                                               9
Headache                                             9
Dizziness                                            8
Chills                                               6
Blood test                                           5
Diarrhoea                                            5
Loss of consciousness                                5
Pyrexia                                              5
Feeling abnormal                                     4
Chest pain                                           4
Bell's palsy                                         4
Cardiac arrest                                       4
Hyperhidrosis                                        4
Asthenia                                             4
Syncope                                              4
Pruritus  